In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import nltk

import re

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
stop_words = set(stopwords.words('english'))

LookupError: ignored

In [ ]:
df = pd.read_csv("Twitter_Data.csv")

In [ ]:
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [ ]:
df.drop(df[df['clean_text'].isna()].index, inplace=True)
df.drop(df[df['category'].isna()].index, inplace=True)

In [ ]:
def preprocess_text(text):
    # Remove non-alphabetic characters and convert to lowercase
    text = re.sub('[^A-Za-z]', ' ', text.lower())
    
    # Tokenize the text
    tokens = text.split()
    
    # Remove stop words
    tokens = [word for word in tokens if word not in stop_words]
    
    # Perform stemming or lemmatization
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    tokens = [stemmer.stem(word) for word in tokens]
    
    # Join the tokens back into a string
    text = ' '.join(tokens)
    
    return text


def rem_unwanted(text):
    text_l = []
    for word in text.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = ''
        
        if word.startswith('#') and len(word) > 1:
            word = ''
            
        elif word.startswith('http'):
            word = ''
        text_l.append(word)

    return " ".join(text_l)


#....................
import string
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def punctuations_rem(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


#....................
def repeating_rem(text):
    return re.sub(r'(.)1+', r'1', text)

In [ ]:
def final_preprocessing(text):
    text = emoji_rem(text)
    text = punctuations_rem(text)
    text = rem_unwanted(text)
    text = repeating_rem(text)
    text = preprocess_text(text)
    text = re.sub(r'[ \n]+', ' ', text)
    
    return text

In [ ]:
df['clean_text']= df['clean_text'].apply(lambda x: preprocess_text(x))

In [ ]:
y = df['category']
X = df['clean_text']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=121)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,hamming_loss,classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
pipe = Pipeline([('tfvec', TfidfVectorizer()), ('nb', RandomForestClassifier())])

In [ ]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('tfvec', TfidfVectorizer()), ('nb', RandomForestClassifier())])

In [ ]:
pipe.score(X_test, y_test)

0.8052606819250987

In [ ]:
pipe.predict(X_test)

array([-1.,  1., -1., ...,  1.,  1.,  1.])